In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pandas as pd
from datetime import date

import bt

import matplotlib.pyplot as plt
from ffn import *
from importlib import reload

In [2]:
# laod file containing buy and sell signals

In [21]:
port_list = pd.read_csv('Portfolio Changes Table 2018-21.csv')

In [22]:
port_list.head()

,Date,Stock Name,Buy/Sell,BB Symbol
0,2018-04-01,Tata Steel Ltd,BUY,TATA IN Equity
1,2018-04-01,JSW Steel Ltd,BUY,JSTL IN Equity
2,2018-04-01,Mphasis,BUY,MPHL IN Equity
3,2018-04-01,Apollo Tyres Ltd.,BUY,APTY IN Equity
4,2018-04-01,Samvardhana Motherson International Ltd.,BUY,MOTHERSO IN Equity


In [23]:
port_list.tail()

,Date,Stock Name,Buy/Sell,BB Symbol
651,2024-10-01,Muthoot Finance Ltd.,BUY,MUTH IN Equity
652,2024-10-01,Zydus Lifesciences Ltd.,BUY,ZYDUSLIF IN Equity
653,2024-10-01,Hindustan Aeronautics Ltd.,BUY,HNAL IN Equity
654,2024-10-01,Ashok Leyland Ltd.,BUY,AL IN Equity
655,2024-10-01,Lupin Ltd.,BUY,LPC IN Equity


In [24]:
port_df = port_list.copy()
#port_df['Date'] = pd.to_datetime(port_df['Date'])

In [25]:
from datetime import timedelta

In [26]:
# Create DataFrames for all 'SELL' and 'BUY' signals for faster access
all_sells = port_df[port_df['Buy/Sell'] == 'SELL'].copy()
all_buys = port_df[port_df['Buy/Sell'] == 'BUY'].copy()

In [27]:
all_sells

,Date,Stock Name,Buy/Sell,BB Symbol
30,2018-06-30,Samvardhana Motherson International Ltd.,SELL,MOTHERSO IN Equity
31,2018-06-30,Coromandel International Ltd,SELL,CRIN IN Equity
32,2018-06-30,Prestige Estates Projects Ltd,SELL,PEPL IN Equity
33,2018-06-30,Gujarat State Petronet Ltd.,SELL,GUJS IN Equity
34,2018-06-30,Mangalore Refinery and Petrochemicals,SELL,MRPL IN Equity
...,...,...,...,...
642,2024-09-30,LIC Housing Finance Ltd.,SELL,LICHF IN Equity
643,2024-09-30,Jindal Steel & Power Ltd.,SELL,JSP IN Equity
644,2024-09-30,Bank Of Baroda,SELL,BOB IN Equity
645,2024-09-30,Tata Steel Ltd.,SELL,TATA IN Equity


In [28]:
all_buys

,Date,Stock Name,Buy/Sell,BB Symbol
0,2018-04-01,Tata Steel Ltd,BUY,TATA IN Equity
1,2018-04-01,JSW Steel Ltd,BUY,JSTL IN Equity
2,2018-04-01,Mphasis,BUY,MPHL IN Equity
3,2018-04-01,Apollo Tyres Ltd.,BUY,APTY IN Equity
4,2018-04-01,Samvardhana Motherson International Ltd.,BUY,MOTHERSO IN Equity
...,...,...,...,...
651,2024-10-01,Muthoot Finance Ltd.,BUY,MUTH IN Equity
652,2024-10-01,Zydus Lifesciences Ltd.,BUY,ZYDUSLIF IN Equity
653,2024-10-01,Hindustan Aeronautics Ltd.,BUY,HNAL IN Equity
654,2024-10-01,Ashok Leyland Ltd.,BUY,AL IN Equity


In [29]:
# First, ensure that the 'Date' columns are in datetime format
all_buys['Date'] = pd.to_datetime(all_buys['Date'])
all_sells['Date'] = pd.to_datetime(all_sells['Date'])

# Get all unique quarter start dates from the buy events
quarter_dates = sorted(all_buys['Date'].unique())

In [30]:
all_buys.groupby('Date')['BB Symbol'].count()

Date
2018-04-01    30
2018-07-01    12
2018-10-01    13
2019-01-01    11
2019-04-01     9
2019-07-01    15
2019-10-01    15
2020-01-01    11
2020-04-01    17
2020-07-01    16
2020-10-01    11
2021-01-01     8
2021-04-01    19
2021-07-01    14
2021-10-01    12
2022-01-01    10
2022-04-01    11
2022-07-01    10
2022-10-01    11
2023-01-01    12
2023-04-01    11
2023-07-01    13
2023-10-01    13
2024-01-01    12
2024-04-01     9
2024-07-01     8
2024-10-01     9
Name: BB Symbol, dtype: int64

In [31]:
# Initialize holdings and portfolio history
holdings = set()
portfolio_history = []

In [32]:
for i, current_date in enumerate(quarter_dates):
    if i == 0:
        # First quarter: select the first 30 securities
        current_buys = all_buys[all_buys['Date'] == current_date]['BB Symbol'].unique()
        # Select only the first 30 securities
        current_buys = current_buys[:30]
        holdings = set(current_buys)
    else:
        previous_date = quarter_dates[i - 1]

        # Remove securities sold between previous and current quarter start dates
        sells_in_period = all_sells[(all_sells['Date'] > previous_date) & (all_sells['Date'] <= current_date)]['BB Symbol'].unique()
        holdings.difference_update(sells_in_period)

        # Add new buys at the current date
        current_buys = all_buys[all_buys['Date'] == current_date]['BB Symbol'].unique()
        holdings.update(current_buys)

        # If holdings exceed 30, remove excess securities
        if len(holdings) > 30:
            holdings = set(list(holdings)[:30])

    # Ensure holdings have exactly 30 securities
    assert len(holdings) == 30, f"Holdings do not have 30 securities at {current_date}"

    # Record the holdings for the current quarter
    portfolio_history.append({
        'Date': current_date,
        'Holdings': list(holdings)
    })

# Create the portfolio DataFrame
portfolio_df = pd.DataFrame(portfolio_history)

In [35]:
# Explode the 'Holdings' column to have one row per security
portfolio_df_exploded = portfolio_df.explode('Holdings').reset_index(drop=True)

# Rename 'Holdings' column to 'BB Symbol'
portfolio_df_exploded.rename(columns={'Holdings': 'BB Symbol'}, inplace=True)

# Create a mapping of 'BB Symbol' to 'Stock Name'
symbols_df = pd.concat([
    all_buys[['BB Symbol', 'Stock Name']],
    all_sells[['BB Symbol', 'Stock Name']]
]).drop_duplicates()

# Merge to get 'Stock Name'
portfolio_df_exploded = portfolio_df_exploded.merge(symbols_df, on='BB Symbol', how='left')

# Rearrange columns
portfolio_df_exploded = portfolio_df_exploded[['Date', 'BB Symbol', 'Stock Name']]

# Sort the DataFrame
portfolio_df_exploded.sort_values(by=['Date', 'BB Symbol'], inplace=True)
portfolio_df_exploded.reset_index(drop=True, inplace=True)

# Display the final DataFrame
print(portfolio_df_exploded)


           Date           BB Symbol                        Stock Name
0    2018-04-01        AL IN Equity                 Ashok Leyland Ltd
1    2018-04-01        AL IN Equity                Ashok Leyland Ltd.
2    2018-04-01      APTY IN Equity                 Apollo Tyres Ltd.
3    2018-04-01      BJHI IN Equity   Bajaj Holdings & Investment Ltd
4    2018-04-01      BJHI IN Equity  Bajaj Holdings & Investment Ltd.
...         ...                 ...                               ...
1050 2024-10-01      SBIN IN Equity              State Bank of India.
1051 2024-10-01      SHFL IN Equity              Shriram Finance Ltd.
1052 2024-10-01      TPWR IN Equity           Tata Power Company Ltd.
1053 2024-10-01      TTMT IN Equity                  Tata Motors Ltd.
1054 2024-10-01  ZYDUSLIF IN Equity           Zydus Lifesciences Ltd.

[1055 rows x 3 columns]


In [38]:
portfolio_df_exploded = portfolio_df_exploded.drop_duplicates(subset=['Date', 'BB Symbol'], keep='last')


In [46]:
portfolio_df_exploded.groupby('Date')['BB Symbol'].count()

Date
2018-04-01    30
2018-07-01    30
2018-10-01    30
2019-01-01    30
2019-04-01    30
2019-07-01    30
2019-10-01    30
2020-01-01    30
2020-04-01    30
2020-07-01    30
2020-10-01    30
2021-01-01    30
2021-04-01    30
2021-07-01    30
2021-10-01    30
2022-01-01    30
2022-04-01    30
2022-07-01    30
2022-10-01    29
2023-01-01    30
2023-04-01    30
2023-07-01    30
2023-10-01    30
2024-01-01    30
2024-04-01    30
2024-07-01    30
2024-10-01    30
Name: BB Symbol, dtype: int64

In [45]:
portfolio_df_exploded = portfolio_df_exploded.dropna(subset=['BB Symbol'])


In [47]:
portfolio_df_exploded[portfolio_df_exploded['Date']=='2022-10-01']

,Date,BB Symbol,Stock Name
718,2022-10-01,ACC IN Equity,ACC Ltd.
719,2022-10-01,AIAE IN Equity,AIA Engineering Ltd.
720,2022-10-01,BHE IN Equity,Bharat Electronics Ltd.
722,2022-10-01,BJAUT IN Equity,Bajaj Auto Ltd.
724,2022-10-01,BJHI IN Equity,Bajaj Holdings & Investment Ltd.
725,2022-10-01,BOS IN Equity,Bosch Ltd.
727,2022-10-01,COAL IN Equity,Coal India Ltd.
729,2022-10-01,CRIN IN Equity,Coromandel International Ltd.
730,2022-10-01,ESCORTS IN Equity,Escorts Kubota Ltd.
732,2022-10-01,FB IN Equity,Federal Bank Ltd.


In [54]:
portfolio_df_exploded['Strat']='Cons'

C:\Users\Amit Kesarkar\AppData\Local\Temp\ipykernel_13096\2358565499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio_df_exploded['Strat']='Cons'


In [55]:
portfolio_df_exploded.head()

,Date,BB Symbol,Stock Name,Strat
1,2018-04-01,AL IN Equity,Ashok Leyland Ltd.,Cons
2,2018-04-01,APTY IN Equity,Apollo Tyres Ltd.,Cons
4,2018-04-01,BJHI IN Equity,Bajaj Holdings & Investment Ltd.,Cons
6,2018-04-01,CCRI IN Equity,Container Corp of India Ltd.,Cons
8,2018-04-01,CRIN IN Equity,Coromandel International Ltd.,Cons


In [56]:
A=portfolio_df_exploded.copy()

In [46]:
#og_port = pd.read_csv('qtr_portfolio_18_21.csv')

In [49]:
#A.to_csv('qtr_portfolio_18_21.csv')

In [58]:
#A = og_port.copy()

In [59]:
A.head()

,Date,BB Symbol,Stock Name,Strat
1,2018-04-01,AL IN Equity,Ashok Leyland Ltd.,Cons
2,2018-04-01,APTY IN Equity,Apollo Tyres Ltd.,Cons
4,2018-04-01,BJHI IN Equity,Bajaj Holdings & Investment Ltd.,Cons
6,2018-04-01,CCRI IN Equity,Container Corp of India Ltd.,Cons
8,2018-04-01,CRIN IN Equity,Coromandel International Ltd.,Cons


In [60]:
A.tail()

,Date,BB Symbol,Stock Name,Strat
1050,2024-10-01,SBIN IN Equity,State Bank of India.,Cons
1051,2024-10-01,SHFL IN Equity,Shriram Finance Ltd.,Cons
1052,2024-10-01,TPWR IN Equity,Tata Power Company Ltd.,Cons
1053,2024-10-01,TTMT IN Equity,Tata Motors Ltd.,Cons
1054,2024-10-01,ZYDUSLIF IN Equity,Zydus Lifesciences Ltd.,Cons


In [46]:
#A = A.drop('Unnamed: 0', axis=1)

In [61]:
A = A.rename(columns={'Date': 'date'})

In [62]:
A['factor_quantile']= A['Strat']
A['Wt'] = A.groupby(['factor_quantile','date'])['BB Symbol'].transform(lambda x:1/len(x))

In [63]:
A

,date,BB Symbol,Stock Name,Strat,factor_quantile,Wt
1,2018-04-01,AL IN Equity,Ashok Leyland Ltd.,Cons,Cons,0.033333
2,2018-04-01,APTY IN Equity,Apollo Tyres Ltd.,Cons,Cons,0.033333
4,2018-04-01,BJHI IN Equity,Bajaj Holdings & Investment Ltd.,Cons,Cons,0.033333
6,2018-04-01,CCRI IN Equity,Container Corp of India Ltd.,Cons,Cons,0.033333
8,2018-04-01,CRIN IN Equity,Coromandel International Ltd.,Cons,Cons,0.033333
...,...,...,...,...,...,...
1050,2024-10-01,SBIN IN Equity,State Bank of India.,Cons,Cons,0.033333
1051,2024-10-01,SHFL IN Equity,Shriram Finance Ltd.,Cons,Cons,0.033333
1052,2024-10-01,TPWR IN Equity,Tata Power Company Ltd.,Cons,Cons,0.033333
1053,2024-10-01,TTMT IN Equity,Tata Motors Ltd.,Cons,Cons,0.033333


In [ ]:
# save base portfolio

In [64]:
A.to_csv('qtr_portfolio_18_21.csv')